In [7]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from collections import defaultdict
import os
import pickle
import scipy.stats as st

from itertools import combinations
from scipy.stats import chi2

from code.algorithm_alpha import algorithm_alpha
from code.algorithm_t import algorithm_t
from scipy.special import factorial

sim = False

In [8]:
df = pd.read_csv('data/df_level2.csv')
df

,CNTSCHID,CNT,PRIVATESCH,STRATIO,SCHSIZE,sum_MATHbelow,mean_ESCS,mean_ESCS_std,Y_MATH,Y_BIN_MATH
0,800004,Albania,public,9.7619,205,11,-1.152958,-0.268234,5,1
1,800006,Albania,public,18.0000,315,15,-1.343747,-0.565595,5,1
2,800007,Albania,public,4.7368,45,2,-2.401967,-2.214922,4,1
3,800008,Albania,private,13.8125,221,6,-0.135900,1.316941,3,1
4,800009,Albania,public,16.9691,823,19,-0.560240,0.655569,2,0
...,...,...,...,...,...,...,...,...,...,...
12615,97500358,B-S-J-Z (China),public,8.0786,1131,0,0.828180,1.595021,0,0
12616,97500359,B-S-J-Z (China),public,11.3202,2298,0,-0.178994,0.251551,0,0
12617,97500360,B-S-J-Z (China),public,9.7932,2605,0,0.190169,0.743977,0,0
12618,97500361,B-S-J-Z (China),public,11.7941,2005,3,-0.929946,-0.750143,0,0


In [9]:
df['STRATIO_scaled'] = preprocessing.scale(df.STRATIO)
df['SCHSIZE_scaled'] = preprocessing.scale(df.SCHSIZE)
df['mean_ESCS_std_scaled'] = preprocessing.scale(df.mean_ESCS_std)

In [10]:
#df.CNT.unique()
df.groupby('CNT').count().reset_index()['CNT']
#df['Y_MATH'].hist()

0                  Albania
1                Argentina
2                Australia
3          B-S-J-Z (China)
4        Baku (Azerbaijan)
5                   Brazil
6        Brunei Darussalam
7                    Chile
8           Chinese Taipei
9                 Colombia
10              Costa Rica
11                 Denmark
12      Dominican Republic
13                 Estonia
14                  France
15                 Georgia
16                  Greece
17               Indonesia
18                 Ireland
19                  Israel
20                   Italy
21                   Japan
22                  Jordan
23              Kazakhstan
24                 Lebanon
25               Lithuania
26              Luxembourg
27                Malaysia
28                   Malta
29                  Mexico
30                 Moldova
31                 Morocco
32                  Panama
33                    Peru
34             Philippines
35                  Poland
36                Portugal
3

In [17]:
ran_var = False
ran_int = True
n_fix = 2
tol = 0.01  # set as 0.01, 0.05 or 0.10 if algorithm_alpha is used; 
            # otherwise use algorithm_t and tune the tolerance tol
model = 'P' # 'B' for Bernoulli and 'P' for Poisson

variab = 'SE'

In [18]:
N = df.CNT.nunique()

lengths = np.array(df.groupby('CNT').count().reset_index().iloc[:,1])

if model == 'B':
    y = list(df.groupby('CNT')['Y_BIN_MATH'].apply(np.array).values)
elif model == 'P':
    y = list(df.groupby('CNT')['Y_MATH'].apply(np.array).values)

fix = defaultdict(list)

if variab == "SE":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
    fix[1] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E
elif variab == "S":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
elif variab == "E":
    fix[0] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E

In [19]:
# with algorithm_alpha

knots, par, W, hess_ran, hess_fix, modeleval = algorithm_alpha(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N, t=None)

1.1575357518191543e-174
1.1575357518191543e-174
1.1575357518191543e-174
1.432502160627851e-91
1.2107611210576881e-73
1.2107611210576881e-73
1.2107611210576881e-73
1.2107611210576881e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2107631882992721e-73
1.2127815843087458e-73
1.2127815843087458e-73
1.2127815843087458e-73
1.2127815843087458e-73
1.2127815843087458e-73
2.0277342445746005e-66
2.0277342445746005e-66
2.0277342445746005e-66
2.0277342445746005e-66
2.0277342445746005e-66
2.0277342445746005e-66
2.0277342445746005e-66
2.0277342445746005e-66
2.0277342445747615e-66
2.0277342445747615e-66
2.0277342445747615e-66
2.0277342445747615e-66
2.0277342

In [20]:
group = np.array([np.nan if np.sum(W[i,:])==0 else np.argmax(W[i,:]) for i in range(N)])

file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + str(variab) + '_' + str(tol) + '_' + str(model) + '_merging' + '.pickle'
f = open('output/case_study_results/' + file_name, 'wb')
pickle.dump([knots, par, W, hess_ran, hess_fix, group, modeleval],f)
f.close()

In [21]:
modeleval[0]

[array([ 1.26431,  1.61904,  1.56268,  0.82221,  0.24073,  2.01203,
         0.12977,  0.61467,  0.30354,  0.62944,  1.06211,  0.48433,
         1.43526,  0.21756,  1.01744,  1.79367, -0.51063,  1.28874,
         2.28047,  2.28043,  0.53419,  0.76894,  0.89404,  0.51045,
         1.63482,  2.02921,  1.62507,  0.32719,  1.79881,  0.56459,
         0.65257,  0.88128,  0.83131,  0.2513 ,  2.27276,  1.65573,
         1.80135,  0.22004,  0.22272,  1.77203,  1.55622,  0.11782,
         1.63986,  2.28071,  1.17206,  1.77416,  0.29422,  0.65025,
         1.76423,  1.78775]),
 array([ 1.22814,  1.6336 ,  1.5037 ,  0.81043,  0.2562 ,  1.92498,
         0.06889,  0.58334,  0.32697,  0.60145,  1.05912,  0.38306,
         1.37673,  0.20645,  0.99099,  1.70664, -0.66721,  1.27349,
         2.26958,  2.2696 ,  0.4943 ,  0.68366,  0.87803,  0.44642,
         1.64867,  1.95616,  1.64019,  0.33374,  1.73307,  0.54925,
         0.63455,  0.85576,  0.81537,  0.27982,  2.26958,  1.66169,
         1.75459, 

In [22]:
[len(array) for array in modeleval[0]]

[50,
 50,
 50,
 50,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37,
 36,
 33,
 32,
 31,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21,
 20,
 19,
 18,
 17,
 16,
 15,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13]

In [23]:
# with algorithm_t

# knots, par, W, hess_ran, hess_fix, modeleval = algorithm_t(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N, t=None)

In [24]:
group = np.array([np.nan if np.sum(W[i,:])==0 else np.argmax(W[i,:]) for i in range(N)])

df.groupby('CNT').count().reset_index()['CNT']

0                  Albania
1                Argentina
2                Australia
3          B-S-J-Z (China)
4        Baku (Azerbaijan)
5                   Brazil
6        Brunei Darussalam
7                    Chile
8           Chinese Taipei
9                 Colombia
10              Costa Rica
11                 Denmark
12      Dominican Republic
13                 Estonia
14                  France
15                 Georgia
16                  Greece
17               Indonesia
18                 Ireland
19                  Israel
20                   Italy
21                   Japan
22                  Jordan
23              Kazakhstan
24                 Lebanon
25               Lithuania
26              Luxembourg
27                Malaysia
28                   Malta
29                  Mexico
30                 Moldova
31                 Morocco
32                  Panama
33                    Peru
34             Philippines
35                  Poland
36                Portugal
3

In [16]:
CI_length = np.array(st.norm.ppf(1 - tol / 2) * np.sqrt(1 / hess_ran))
pd.DataFrame({'lb': knots - CI_length.flatten(), 'ub': knots + CI_length.flatten()})

,lb,ub
0,-0.909725,-0.213219
1,-3.209331,-2.470325
2,1.782470,2.685237
3,-4.641562,-2.418206
4,0.209337,0.666958
5,-1.158109,-0.804384
6,-2.311653,-1.846308
7,0.934151,1.567852
8,0.837481,1.642361
9,-2.851021,-2.137573
